In [1]:
pip install --upgrade pip

  Attempting uninstall: pip
    Found existing installation: pip 21.0.1
    Uninstalling pip-21.0.1:
      Successfully uninstalled pip-21.0.1
Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install opencage

Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install geopy

Note: you may need to restart the kernel to use updated packages.


In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn
from opencage.geocoder import OpenCageGeocode
from pprint import pprint
from geopy import distance
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
pd.set_option('display.max_columns', 300) # allow us read the full picture of dataframe

In [2]:
turn = pd.read_csv("dataset/Turnaround_overview_report_EPS_PGT_ALL.csv", 
                   parse_dates=['Decision Date','Response Date','A1 Created date','A2 Created date'],infer_datetime_format=True)
applist = pd.read_csv("dataset/ApplicationList_Qualifications_EPS_PGT_ALL.csv", parse_dates=['Latest Decision Date'],infer_datetime_format=True)
reg = pd.read_csv("dataset/pgt_registrations-2022-11-01.csv")
eng = pd.read_csv("dataset/English Results report_EPS_PGT_ALL1.csv")


## **1. Check the Application List dataframe**
This dataframe contains the information of the applicant.

In [3]:
applist.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15643 entries, 0 to 15642
Data columns (total 28 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   Year of Entry           15643 non-null  object        
 1   College                 15643 non-null  object        
 2   Department              15643 non-null  object        
 3   Applicant ID            15643 non-null  int64         
 4   City - Address line 4   15633 non-null  object        
 5   Fee Status              15643 non-null  object        
 6   Level                   15643 non-null  object        
 7   Programme               15643 non-null  object        
 8   Latest Decision         15640 non-null  object        
 9   Latest Decision Date    15270 non-null  datetime64[ns]
 10  Country of Birth        15626 non-null  object        
 11  Domicile                15570 non-null  object        
 12  Nationality             15626 non-null  object

In [4]:
applist.head()

,Year of Entry,College,Department,Applicant ID,City - Address line 4,Fee Status,Level,Programme,Latest Decision,Latest Decision Date,Country of Birth,Domicile,Nationality,Degree,Institution code,Institution Name clean,Year of graduation,Classification,Prof qual,Prof qual desc,Masters,Masters desc,qual descriptions,UoB?,Student ID,Country of Institution,Grade on application,name of institution
0,2021/22,EPS,COMPUTER SCI,2305538,BENGALURU,Overseas,PGT,MSc Computer Science FT,Conditional Offer - waiting for response,2021-02-25,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12TH,No,Active,IndiaIndia,8.4,PES University
1,2021/22,EPS,BUSINESS,2376697,KHAJRANA INDORE,Overseas,PGT,MSc Adv Eng Man (Constr Man)FT,Conditional Offer - waiting for response,2021-06-10,India,India,India,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12TH GRADE,No,Active,IndiaIndia,7.42 CGPA,ACROPOLIS INSTITUTE OF TECHNOLOGY & RESEARCH
2,2021/22,EPS,COMPUTER SCI,2374445,"BARWAHA , KHARGONE",Overseas,PGT,MSc Artific Int & Mach Lear FT,Rejected at stage 1,2021-06-25,India,India,India,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12TH GRADE,No,Active,IndiaIndia,73.50%,"AISSM'S Institute of Information Technology, Pune"
3,2021/22,EPS,EEC ENG,2263436,CHANGCHUN CITY,Overseas,PGT,MSc Elec + Computer Eng FT,Withdrawn,2020-11-06,China,China,China,NaN,Z0036,Guizhou University,NaN,NaN,NaN,NaN,NaN,NaN,NaN,No,Withdrawn,China,79.13%,Guizhou University
4,2021/22,EPS,COMPUTER SCI,2295129,"WEIHAI, SHANDONG",Overseas,PGT,MSc Artific Int & Mach Lear FT,Conditional Offer - waiting for response,2021-03-18,China,China,China,NaN,Z0091,Shandong University,NaN,NaN,NaN,NaN,NaN,NaN,NaN,No,Active,ChinaChina,,Shandong University


### 1.drop the columns with values couldn't be numeric or normalized

In [5]:
applist = applist.drop(columns=['Year of Entry','College','Level', 'Degree', 'Year of graduation', 'Prof qual', 
                                'Prof qual desc','Masters desc', 'Student ID'])
applist.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15643 entries, 0 to 15642
Data columns (total 19 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   Department              15643 non-null  object        
 1   Applicant ID            15643 non-null  int64         
 2   City - Address line 4   15633 non-null  object        
 3   Fee Status              15643 non-null  object        
 4   Programme               15643 non-null  object        
 5   Latest Decision         15640 non-null  object        
 6   Latest Decision Date    15270 non-null  datetime64[ns]
 7   Country of Birth        15626 non-null  object        
 8   Domicile                15570 non-null  object        
 9   Nationality             15626 non-null  object        
 10  Institution code        9747 non-null   object        
 11  Institution Name clean  9747 non-null   object        
 12  Classification          774 non-null    object

### 2.transform the object columns with binary code.
For the 'uob_alum', uob alumni =1, non-alumni =0. \
For the 'home_oversea', oversea = 1, home = 0

In [6]:
transform = LabelEncoder()
applist['uob_alum'] = transform.fit_transform(applist['UoB?'])
applist['home_oversea'] = transform.fit_transform(applist['Fee Status'])
applist = applist.drop(['UoB?','Fee Status'], axis=1)

For the 'master_prev', applicant has previous master = 1, non-master = 0. 

In [7]:
applist['Masters'].unique()

array([nan, 'Y', 'YY', 'YYYY', 'YYYYYY', 'YYY', 'YYYYYYY', 'YYYYYYYYY',
       'YYYYYYYYYYYY', 'YYYYY', 'YYYYYYYY'], dtype=object)

In [8]:
applist['master_prev'] = ''
def master(x):
    if pd.isna(x['Masters']):
        return 0
    else:
        return 1
applist.loc[:,'master_prev'] = applist.apply(master,axis=1)
applist = applist.drop('Masters', axis=1)

### 3.Calculate the distance for the **home** student. 
Assumption: for the oversea student the distance between home and birmingham won't be a factor because they already consider it before applying. Therefore, all the oversea student would be given a distance of 8000

In [9]:
# find the geography data of birmingham
key = '0f33c14b29bf4e89866304be3e0646af'

geocoder = OpenCageGeocode(key)
query = 'birmingham, uk'
results = geocoder.geocode(query)
lat = results[0]['geometry']['lat']
lng = results[0]['geometry']['lng']
print (lat, lng)

52.4796992 -1.9026911


In [10]:
#There is only 10 nans in the city column, the forward fill is acceptable
applist["City - Address line 4"] = applist["City - Address line 4"].fillna(method="ffill")

In [11]:
# a function to calculate the distance for the home student
def find_distance(x):
    key = '0f33c14b29bf4e89866304be3e0646af'
    geocoder = OpenCageGeocode(key)
    if x['home_oversea'] == 0:
        result_A = geocoder.geocode(x["City - Address line 4"])
        if result_A != []:
            lat_A = result_A[0]['geometry']['lat']
            lng_A = result_A[0]['geometry']['lng']    
            return (distance.distance((lat_A,lng_A), (52.4796992, -1.9026911)).kilometers)
        else: 
            return 0
    else: 
        return 8000

applist['distance'] = ''
applist.loc[:,"distance"] = applist.apply(find_distance,axis=1)

In [12]:
applist.head()

,Department,Applicant ID,City - Address line 4,Programme,Latest Decision,Latest Decision Date,Country of Birth,Domicile,Nationality,Institution code,Institution Name clean,Classification,qual descriptions,Country of Institution,Grade on application,name of institution,uob_alum,home_oversea,master_prev,distance
0,COMPUTER SCI,2305538,BENGALURU,MSc Computer Science FT,Conditional Offer - waiting for response,2021-02-25,NaN,NaN,NaN,NaN,NaN,NaN,12TH,IndiaIndia,8.4,PES University,0,1,0,8000.0
1,BUSINESS,2376697,KHAJRANA INDORE,MSc Adv Eng Man (Constr Man)FT,Conditional Offer - waiting for response,2021-06-10,India,India,India,NaN,NaN,NaN,12TH GRADE,IndiaIndia,7.42 CGPA,ACROPOLIS INSTITUTE OF TECHNOLOGY & RESEARCH,0,1,0,8000.0
2,COMPUTER SCI,2374445,"BARWAHA , KHARGONE",MSc Artific Int & Mach Lear FT,Rejected at stage 1,2021-06-25,India,India,India,NaN,NaN,NaN,12TH GRADE,IndiaIndia,73.50%,"AISSM'S Institute of Information Technology, Pune",0,1,0,8000.0
3,EEC ENG,2263436,CHANGCHUN CITY,MSc Elec + Computer Eng FT,Withdrawn,2020-11-06,China,China,China,Z0036,Guizhou University,NaN,NaN,China,79.13%,Guizhou University,0,1,0,8000.0
4,COMPUTER SCI,2295129,"WEIHAI, SHANDONG",MSc Artific Int & Mach Lear FT,Conditional Offer - waiting for response,2021-03-18,China,China,China,Z0091,Shandong University,NaN,NaN,ChinaChina,,Shandong University,0,1,0,8000.0


#### <font color='red'>Since the distance function takes a long time to execute, here we save this intermediate dataframe to pickle to improve efficiency for subsequent use</font>

In [53]:
applist.to_pickle("df/applist.pkl")  

In [ ]:
applist = pd.read_pickle("df/applist.pkl")

Save to csv is also okay for temporary storage but it would generate one more index column. Therefore pickle is better.\
The following code is no longer used\
#applist.to_csv('applist.csv')
#applist = pd.read_csv("applist.csv")
#applist = applist.drop(['Unnamed: 0'], axis=1)

### 4. Process the Institution data.
#### Fill the blank in 'Institution Name clean' with 'name of institution'
Reason: in the raw dataset, the 'Institution Name clean' field is from a Drop-down menu of the application system, thus more accurate and consistent, but many Nan values. The 'name of institution' is a field the applicant input the text, more casual and a same university may be under different names

In [14]:
applist['Institution'] = ''
def insti(x):
    if pd.isna(x['Institution Name clean']):
        return x['name of institution']
    else:
        return x['Institution Name clean']
applist.loc[:,'Institution'] = applist.apply(insti,axis=1)
applist = applist.drop('Institution Name clean', axis=1)

#### Count the times an Institution appears in the application list

In [15]:
group_by_insti = applist.groupby(['Institution'])
insti_app_count = group_by_insti['Applicant ID'].count().reset_index(name='Institution_app_num')
insti_app_count.nunique()

Institution            5112
Institution_app_num      58
dtype: int64

In [16]:
applist = applist.merge(insti_app_count, how='left', on='Institution')
applist = applist.drop(columns=['Institution code','Classification','name of institution','qual descriptions'])
applist.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 15643 entries, 0 to 15642
Data columns (total 17 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   Department              15643 non-null  object        
 1   Applicant ID            15643 non-null  int64         
 2   City - Address line 4   15643 non-null  object        
 3   Programme               15643 non-null  object        
 4   Latest Decision         15640 non-null  object        
 5   Latest Decision Date    15270 non-null  datetime64[ns]
 6   Country of Birth        15626 non-null  object        
 7   Domicile                15570 non-null  object        
 8   Nationality             15626 non-null  object        
 9   Country of Institution  15609 non-null  object        
 10  Grade on application    15643 non-null  object        
 11  uob_alum                15643 non-null  int32         
 12  home_oversea            15643 non-null  int32 

## **2. Check the Turn Around dataframe**
This dataframe contains the information of the timeline. 

In [17]:
turn = pd.read_csv("dataset/Turnaround_overview_report_EPS_PGT_ALL.csv", 
                   parse_dates=['Decision Date','Response Date','A1 Created date', 'A1 Updated date','A2 Created date','A2 Updated date'],infer_datetime_format=True)
print(list(turn))

['Applicant ID', 'Seq', 'Received Date', 'Fee Status', 'Programme', 'School', 'Workflow Stage', 'In touch with Supervisor?', 'Decision', 'Decision Date', 'Response', 'Response Date', 'Confirmation', 'Conf. Date', 'Reject Reason', 'A1 Created date', 'A1 Updated date', 'Sc Hub/Tutor Created date', 'Sc Hub/Tutor Updated date', 'Sc Supervisor Created date', 'Sc Supervisor Updated date', 'Sc Sign-off Created date', 'Sc Sign-off Updated date', 'A2 Created date', 'A2 Updated date', 'Funding obtained?', 'IDRC', 'Year of entry', 'College', 'WF code', 'App Location', 'On Hold', 'Funding', 'Days at Admissions Review', 'Days in School', 'Days at Decision Stage', 'Total Days Admissions', 'TOTAL Days', 'Response days', 'Sep_01', 'decision_since_Sep01', 'A1C_since_Sep01', 'A1U_since_Sep01', 'A2C_since_Sep01', 'Response_since_Sep01']


In [18]:
turn.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15641 entries, 0 to 15640
Data columns (total 45 columns):
 #   Column                      Non-Null Count  Dtype         
---  ------                      --------------  -----         
 0   Applicant ID                15641 non-null  int64         
 1   Seq                         15641 non-null  int64         
 2   Received Date               15641 non-null  object        
 3   Fee Status                  15641 non-null  object        
 4   Programme                   15641 non-null  object        
 5   School                      14244 non-null  object        
 6   Workflow Stage              15641 non-null  object        
 7   In touch with Supervisor?   2 non-null      object        
 8   Decision                    15271 non-null  object        
 9   Decision Date               15270 non-null  datetime64[ns]
 10  Response                    4265 non-null   object        
 11  Response Date               4265 non-null   datetime64

In [19]:
turn.head()

,Applicant ID,Seq,Received Date,Fee Status,Programme,School,Workflow Stage,In touch with Supervisor?,Decision,Decision Date,Response,Response Date,Confirmation,Conf. Date,Reject Reason,A1 Created date,A1 Updated date,Sc Hub/Tutor Created date,Sc Hub/Tutor Updated date,Sc Supervisor Created date,Sc Supervisor Updated date,Sc Sign-off Created date,Sc Sign-off Updated date,A2 Created date,A2 Updated date,Funding obtained?,IDRC,Year of entry,College,WF code,App Location,On Hold,Funding,Days at Admissions Review,Days in School,Days at Decision Stage,Total Days Admissions,TOTAL Days,Response days,Sep_01,decision_since_Sep01,A1C_since_Sep01,A1U_since_Sep01,A2C_since_Sep01,Response_since_Sep01
0,2305538,1,2020-11-30,O,MSc Computer Science FT,Computer Science,Initial decision complete: awaiting app. accept,NaN,C,2021-02-25,NaN,NaT,NaN,NaN,Clearance Checks Only,2020-11-30,2021-01-28,2021-01-28,2021-02-05,NaN,NaN,NaN,NaN,2021-02-05,2021-02-25,N,C,2021/22,EPS,RD_AA,Decision Entered,NaN,\r\n\r\n\r\n\r\n- Personal or family funds\r\n,59,8,20,79,87,0,2020-09-01,177,90,149,157,0
1,2376697,1,2021-05-24,O,MSc Adv Eng Man (Constr Man)FT,NaN,Initial decision complete: awaiting app. accept,NaN,C,2021-06-10,NaN,NaT,NaN,NaN,NaN,2021-05-24,2021-06-10,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT,N,C,2021/22,EPS,RD_AA,Decision Entered,NaN,\r\n\r\n\r\n\r\n- Personal or family funds\r\n,17,0,0,17,17,0,2020-09-01,282,265,282,0,0
2,2374445,1,2021-05-12,O,MSc Artific Int & Mach Lear FT,Computer Science,Applicant has been Rejected,NaN,R,2021-06-25,NaN,NaT,NaN,NaN,Rejection - Doesn't Meet Entry Requirements,2021-05-12,2021-06-25,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT,N,R,2021/22,EPS,RD_EN,Rejected,NaN,\r\n\r\n\r\n\r\n- Personal or family funds\r\n,44,0,0,44,44,0,2020-09-01,297,253,297,0,0
3,2263436,1,2020-10-06,O,MSc Elec + Computer Eng FT,Elec Elec and System Eng;Elec Elec and Sys Eng...,Withdrawn,NaN,C,2020-11-06,A,2006-11-20,NaN,NaN,NaN,2020-10-06,2020-11-06,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT,N,CA,2021/22,EPS,WD_WD,Withdrawn,NaN,\r\n\r\n\r\n\r\n- Personal or family funds\r\n,31,0,0,31,31,0,2020-09-01,66,35,66,0,66
4,2295129,2,2020-11-16,O,MSc Artific Int & Mach Lear FT,Computer Science,Initial decision complete: awaiting app. accept,NaN,C,2021-03-18,NaN,NaT,NaN,NaN,Final Grade Average 80%,2020-11-16,2021-02-15,2021-02-15,2021-02-15,NaN,NaN,NaN,NaN,2021-02-15,2021-03-18,N,C,2021/22,EPS,RD_AA,Decision Entered,NaN,\r\n\r\n\r\n\r\n- Personal or family funds\r\n,91,0,31,122,122,0,2020-09-01,198,76,167,167,0


### 1. only keep the columns with useful value

In [20]:
turn = turn.loc[:,['Applicant ID',  'Fee Status', 'Programme', 'School', 'Decision','Funding obtained?',
              'A1 Created date','A1 Updated date', 'A2 Created date', 'Decision Date','Response Date',
             'Days at Admissions Review', 'Days in School', 'Days at Decision Stage', 'Total Days Admissions', 'TOTAL Days', 'Response days',
             'decision_since_Sep01', 'A1C_since_Sep01', 'A1U_since_Sep01', 'A2C_since_Sep01', 'Response_since_Sep01']]

# based on the communication with Admission Office, those who doesn't have the 'School' value should be 'Business School'
turn["School"] = turn["School"].fillna('Business School')

### 2. Process the 'funding' column to binary code

In [21]:
turn['funding'] = ''

def funding(x):
    if x["Funding obtained?"] == 'Y':
        return 1
    else:
        return 0

turn.loc[:,'funding'] = turn.apply(funding,axis=1)
turn = turn.drop(['Funding obtained?'], axis=1)

### 3. divide the 'Decision' column into 3 columns with One Hot Encoder

In [22]:
turn['Decision'] = turn['Decision'].fillna(method="ffill")
turn['Decision'].unique()

array(['C', 'R', 'U'], dtype=object)

In [23]:
ohe = OneHotEncoder(sparse=False)
turn[['C', 'R', 'U']] = ohe.fit_transform(turn['Decision'].values.reshape(-1,1))
print(ohe.get_feature_names(['Decision']))

['Decision_C' 'Decision_R' 'Decision_U']


In [24]:
turn.rename(columns={'C': 'Decision_C', 'R': 'Decision_R','U': 'Decision_U'}, inplace=True)
turn = turn.drop(['Decision'], axis=1)
turn= turn.fillna(method="ffill")
turn.head()

,Applicant ID,Fee Status,Programme,School,A1 Created date,A1 Updated date,A2 Created date,Decision Date,Response Date,Days at Admissions Review,Days in School,Days at Decision Stage,Total Days Admissions,TOTAL Days,Response days,decision_since_Sep01,A1C_since_Sep01,A1U_since_Sep01,A2C_since_Sep01,Response_since_Sep01,funding,Decision_C,Decision_R,Decision_U
0,2305538,O,MSc Computer Science FT,Computer Science,2020-11-30,2021-01-28,2021-02-05,2021-02-25,NaT,59,8,20,79,87,0,177,90,149,157,0,0,1.0,0.0,0.0
1,2376697,O,MSc Adv Eng Man (Constr Man)FT,Business School,2021-05-24,2021-06-10,2021-02-05,2021-06-10,NaT,17,0,0,17,17,0,282,265,282,0,0,0,1.0,0.0,0.0
2,2374445,O,MSc Artific Int & Mach Lear FT,Computer Science,2021-05-12,2021-06-25,2021-02-05,2021-06-25,NaT,44,0,0,44,44,0,297,253,297,0,0,0,0.0,1.0,0.0
3,2263436,O,MSc Elec + Computer Eng FT,Elec Elec and System Eng;Elec Elec and Sys Eng...,2020-10-06,2020-11-06,2021-02-05,2020-11-06,2006-11-20,31,0,0,31,31,0,66,35,66,0,66,0,1.0,0.0,0.0
4,2295129,O,MSc Artific Int & Mach Lear FT,Computer Science,2020-11-16,2021-02-15,2021-02-15,2021-03-18,2006-11-20,91,0,31,122,122,0,198,76,167,167,0,0,1.0,0.0,0.0


### 4. merge the processed application list and the turn around dataframe 

In [25]:
dataframe = applist.merge(turn, how='left', on=['Applicant ID','Programme'])
dataframe = dataframe.drop_duplicates(subset=['Applicant ID','Programme'], keep='last')
dataframe.shape

(15604, 39)

## **3. Check the English dataframe**
This dataframe contains information of the english test

In [26]:
eng = pd.read_csv("dataset/English Results report_EPS_PGT_ALL1.csv")
print(list(eng))

['Application Received', 'Applicant ID', 'Start Date', 'Programme', 'Status', 'Decision', 'Results Overall', 'Listening', 'Reading', 'Writing', 'Speaking', 'Qual_Name', 'Date Taken', 'Nationality', 'Qual_Name_Other', 'Date taken other', 'Other_Num', 'Other_read', 'Other_writ', 'Other_listening', 'Other_speak', 'Other_total', 'Unnamed: 22']


In [27]:
eng.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15643 entries, 0 to 15642
Data columns (total 23 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Application Received  15643 non-null  object 
 1   Applicant ID          15643 non-null  int64  
 2   Start Date            15604 non-null  object 
 3   Programme             15643 non-null  object 
 4   Status                15643 non-null  object 
 5   Decision              15166 non-null  object 
 6   Results Overall       5921 non-null   float64
 7   Listening             5325 non-null   float64
 8   Reading               5325 non-null   float64
 9   Writing               5325 non-null   float64
 10  Speaking              5325 non-null   float64
 11  Qual_Name             4909 non-null   object 
 12  Date Taken            4909 non-null   object 
 13  Nationality           15626 non-null  object 
 14  Qual_Name_Other       644 non-null    object 
 15  Date taken other   

### 1. drop the duplicate rows and columns with too few values

In [28]:
# drop the duplicate rows and columns with too few values
eng= eng.drop_duplicates()
eng = eng.drop(columns=['Application Received','Start Date', 'Decision', 'Nationality', 'Qual_Name_Other',
 'Date taken other', 'Other_Num', 'Other_read', 'Other_writ', 'Other_listening', 'Other_speak', 'Other_total', 
  'Unnamed: 22','Date Taken'])   

### 2. Fill some blanks where we know what it should be
For those record having a result but no Qual_Name, I check in the excel and turns out that they are all IELTS test

In [29]:
condi = eng['Results Overall'] >= 0
eng.loc[condi,'Qual_Name'] = eng.loc[condi,'Qual_Name'].fillna('IELTS (International English Language Testing System)')
eng.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 15641 entries, 0 to 15642
Data columns (total 9 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Applicant ID     15641 non-null  int64  
 1   Programme        15641 non-null  object 
 2   Status           15641 non-null  object 
 3   Results Overall  5920 non-null   float64
 4   Listening        5324 non-null   float64
 5   Reading          5324 non-null   float64
 6   Writing          5324 non-null   float64
 7   Speaking         5324 non-null   float64
 8   Qual_Name        5984 non-null   object 
dtypes: float64(5), int64(1), object(3)
memory usage: 1.2+ MB


In [30]:
eng['Qual_Name'].unique()

array([nan, 'IELTS (International English Language Testing System)',
       'TOEFL', 'IELTS (International English Language Testing Syst',
       'Pearson Academic English Test',
       'TOEFLIELTS (International English Language Testing Syst',
       'IELTS (International English Language Testing System)Pearson Academic English Test',
       'TOEFLIELTS (International English Language Testing System)',
       'TOEFLPearson Academic English Test'], dtype=object)

### 3. distribute the three types of english test score into the corresponding column

In [31]:
# distribute the three types of english test score
ielts = ['IELTS (International English Language Testing System)','IELTS (International English Language Testing Syst',
        'IELTS (International English Language Testing System)Pearson Academic English Test']
toefl = ['TOEFL','TOEFLIELTS (International English Language Testing Syst',
       'TOEFLIELTS (International English Language Testing System)',
       'TOEFLPearson Academic English Test']
pte = ['Pearson Academic English Test']

# IELTS
def ielts_o(x):
    if x['Qual_Name'] in ielts:
        return x['Results Overall']
    else:
        return 0
eng.loc[:,'Ielts_Overall'] = eng.apply(ielts_o,axis=1)

def ielts_l(x):
    if x['Qual_Name'] in ielts:
        return x['Listening']
    else:
        return 0
eng.loc[:,'Ielts_Listening'] = eng.apply(ielts_l,axis=1)    
    
def ielts_r(x):
    if x['Qual_Name'] in ielts:
        return x['Reading']
    else:
        return 0
eng.loc[:,'Ielts_Reading'] = eng.apply(ielts_r,axis=1)       
    
def ielts_w(x):
    if x['Qual_Name'] in ielts:
        return x['Writing']
    else:
        return 0
eng.loc[:,'Ielts_Writing'] = eng.apply(ielts_w,axis=1)       
    
def ielts_s(x):
    if x['Qual_Name'] in ielts:
        return x['Speaking']
    else:
        return 0
eng.loc[:,'Ielts_Speaking'] = eng.apply(ielts_s,axis=1)       

# TOEFL
def toefl_o(x):
    if x['Qual_Name'] in toefl:
        return x['Results Overall']
    else:
        return 0
eng.loc[:,'Toefl_Overall'] = eng.apply(toefl_o,axis=1)       

def toefl_l(x):
    if x['Qual_Name'] in toefl:
        return x['Listening']
    else:
        return 0
eng.loc[:,'Toefl_Listening'] = eng.apply(toefl_l,axis=1)    
    
def toefl_r(x):
    if x['Qual_Name'] in toefl:
        return x['Reading']
    else:
        return 0
eng.loc[:,'Toefl_Reading'] = eng.apply(toefl_r,axis=1)       
    
def toefl_w(x):
    if x['Qual_Name'] in toefl:
        return x['Writing']
    else:
        return 0
eng.loc[:,'Toefl_Writing'] = eng.apply(toefl_w,axis=1)       
    

def toefl_s(x):
    if x['Qual_Name'] in toefl:
        return x['Speaking']
    else:
        return 0
eng.loc[:,'Toefl_Speaking'] = eng.apply(toefl_s,axis=1)       

# PTE
def pte_o(x):
    if x['Qual_Name'] in pte:
        return x['Results Overall']
    else:
        return 0
eng.loc[:,'PTE_Overall'] = eng.apply(pte_o,axis=1)    

def pte_l(x):
    if x['Qual_Name'] in pte:
        return x['Listening']
    else:
        return 0
eng.loc[:,'PTE_Listening'] = eng.apply(pte_l,axis=1)    
    
def pte_r(x):
    if x['Qual_Name'] in pte:
        return x['Reading']
    else:
        return 0
eng.loc[:,'PTE_Reading'] = eng.apply(pte_r,axis=1)       
    
def pte_w(x):
    if x['Qual_Name'] in pte:
        return x['Writing']
    else:
        return 0
eng.loc[:,'PTE_Writing'] = eng.apply(pte_w,axis=1)       
    

def pte_s(x):
    if x['Qual_Name'] in pte:
        return x['Speaking']
    else:
        return 0
eng.loc[:,'PTE_Speaking'] = eng.apply(pte_s,axis=1)           
    
eng = eng.drop(['Results Overall', 'Listening', 'Reading', 'Writing', 'Speaking'], axis=1)
eng.head()

,Applicant ID,Programme,Status,Qual_Name,Ielts_Overall,Ielts_Listening,Ielts_Reading,Ielts_Writing,Ielts_Speaking,Toefl_Overall,Toefl_Listening,Toefl_Reading,Toefl_Writing,Toefl_Speaking,PTE_Overall,PTE_Listening,PTE_Reading,PTE_Writing,PTE_Speaking
0,2305538,MSc Computer Science FT,Active,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2376697,MSc Adv Eng Man (Constr Man)FT,Active,IELTS (International English Language Testing ...,8.0,8.0,9.0,6.5,7.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2374445,MSc Artific Int & Mach Lear FT,Active,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2263436,MSc Elec + Computer Eng FT,Withdrawn,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2295129,MSc Computer Science FT,Withdrawn,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### 4. change the Qual_Name into code

In [32]:
def eng_type(x):
    if x['Qual_Name'] in ielts:
        return 1
    elif x['Qual_Name'] in toefl:
        return 2
    elif x['Qual_Name'] in pte:
        return 3
    else:
        return 0
eng.loc[:,'eng_type'] = eng.apply(eng_type,axis=1)

In [33]:
eng.describe()

,Applicant ID,Ielts_Overall,Ielts_Listening,Ielts_Reading,Ielts_Writing,Ielts_Speaking,Toefl_Overall,Toefl_Listening,Toefl_Reading,Toefl_Writing,Toefl_Speaking,PTE_Overall,PTE_Listening,PTE_Reading,PTE_Writing,PTE_Speaking,eng_type
count,1.564100e+04,15591.000000,14993.000000,14993.000000,14993.000000,14993.000000,15627.000000,15626.000000,15626.000000,15626.000000,15626.000000,15641.000000,15641.000000,15641.000000,15641.000000,15641.000000,15641.000000
mean,2.291818e+06,2.126631,2.170013,2.150570,1.906023,1.942740,4.450547,1.183092,1.214770,1.106809,1.050077,0.290359,0.287833,0.292373,0.301963,0.307014,0.438015
std,1.350910e+05,4.341546,3.566692,3.543283,3.172329,3.199135,20.453754,5.643243,5.776929,5.310296,5.018156,4.515932,4.451846,4.524556,4.666765,4.769305,0.604658
min,3.305240e+05,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,2.277702e+06,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,2.313898e+06,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,2.362120e+06,6.000000,6.000000,6.000000,5.500000,5.500000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
max,2.533710e+06,112.000000,68.000000,68.000000,73.000000,72.000000,193.000000,120.000000,118.000000,119.000000,117.000000,90.000000,90.000000,90.000000,90.000000,90.000000,3.000000


### 5. Merge the english into the whole dataframe

In [34]:
dataframe = dataframe.merge(eng, how='left', on=['Applicant ID','Programme'])
dataframe = dataframe.drop_duplicates(subset=['Applicant ID','Programme'], keep='last')
dataframe.shape

(15604, 57)

## **4. Check the dataframe now**
### Find out the applicants have multiple applications

In [35]:
group_by_id = dataframe.groupby(['Applicant ID'])
multi_app_count = group_by_id['Applicant ID'].count().reset_index(name='multi_app')
multi_app_count.nunique()

Applicant ID    14495
multi_app           5
dtype: int64

In [36]:
dataframe = dataframe.merge(multi_app_count, how='left', on='Applicant ID')
dataframe.shape

(15604, 58)

## **5. Check the registration dataframe**
This dataframe contains information of the students that register in this academic year

In [37]:
reg = pd.read_csv("dataset/pgt_registrations-2022-11-01.csv")
reg.head()

,Applicant ID,Prog Code,Prog Desc,Date Started
0,1254129,526C,M.Sc. (Taught) Railway Systems Engineering & I...,30-Aug-21
1,1902175,6447,M.Sc. (Taught) Rail Systems Engineering and In...,30-Aug-21
2,2064078,526C,M.Sc. (Taught) Railway Systems Engineering & I...,30-Aug-21
3,2192896,526C,M.Sc. (Taught) Railway Systems Engineering & I...,30-Aug-21
4,2296477,526C,M.Sc. (Taught) Railway Systems Engineering & I...,30-Aug-21


In [38]:
reg.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1302 entries, 0 to 1301
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Applicant ID  1302 non-null   int64 
 1   Prog Code     1302 non-null   object
 2   Prog Desc     1302 non-null   object
 3   Date Started  1302 non-null   object
dtypes: int64(1), object(3)
memory usage: 40.8+ KB


### Merge the registration into the whole dataframe

In [39]:
dataframe = dataframe.merge(reg, how='left', on='Applicant ID')
dataframe = dataframe.drop_duplicates(subset=['Applicant ID','Programme'], keep='last')
dataframe.shape

(15604, 61)

## **6. Find the situation of the applicant**
### 1.Find out whether the applicant get the offer.
Due to some errors in the data set, there are three possible conditions when the the applicant was offered:\
Decision = C\
Decision = U\
Their IDs are on the registration dataframe.

In [40]:
dataframe['offered'] = ''

def offered(x):
    if x["Decision_C"] == 1:
        return 1
    elif x["Decision_U"] == 1:
        return 1
    else:
        if pd.isna(x["Prog Code"]):
            return 0
        else:
            return 1

dataframe.loc[:,'offered'] = dataframe.apply(offered,axis=1)

### 2. Find out the applicants received multiple offers

In [41]:
offer = dataframe.loc[dataframe["offered"] == 1]
group_by_offer = offer.groupby(['Applicant ID'])
multi_offer_count = group_by_offer['Applicant ID'].count().reset_index(name='offer_num')
multi_offer_count.nunique()

Applicant ID    8452
offer_num          5
dtype: int64

In [42]:
dataframe = dataframe.merge(multi_offer_count, how='left', on='Applicant ID')
dataframe['offer_num'].fillna(0)

0        1.0
1        1.0
2        0.0
3        1.0
4        2.0
        ... 
15599    0.0
15600    0.0
15601    1.0
15602    2.0
15603    0.0
Name: offer_num, Length: 15604, dtype: float64

In [43]:
dataframe.shape

(15604, 63)

### 3.Find out the applicant accept the offer, add the tag of offer acceptance(whose ID in the registration）

In [44]:
dataframe['accepted'] = ''

def accepted(x):
    if pd.isna(x["Prog Code"]):
        return 0
    else:
        return 1

dataframe.loc[:,'accepted'] = dataframe.apply(accepted,axis=1)

## **7. Count the institution frequency of the offer**

In [45]:
group_by_offer = offer.groupby(['Institution'])
insti_offer_count = group_by_offer['Applicant ID'].count().reset_index(name='Institution_offer_num')
insti_offer_count.nunique()

Institution              3266
Institution_offer_num      39
dtype: int64

In [46]:
dataframe = dataframe.merge(insti_offer_count, how='left', on='Institution')
dataframe['Institution_offer_num'].fillna(0)
dataframe.shape

(15604, 65)

In [47]:
dataframe['Insti_weight'] = ''
dataframe.eval('Insti_weight = Institution_offer_num / Institution_app_num', inplace=True)

In [48]:
dataframe.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 15604 entries, 0 to 15603
Data columns (total 66 columns):
 #   Column                     Non-Null Count  Dtype         
---  ------                     --------------  -----         
 0   Department                 15604 non-null  object        
 1   Applicant ID               15604 non-null  int64         
 2   City - Address line 4      15604 non-null  object        
 3   Programme                  15604 non-null  object        
 4   Latest Decision            15601 non-null  object        
 5   Latest Decision Date       15238 non-null  datetime64[ns]
 6   Country of Birth           15587 non-null  object        
 7   Domicile                   15531 non-null  object        
 8   Nationality                15587 non-null  object        
 9   Country of Institution     15570 non-null  object        
 10  Grade on application       15604 non-null  object        
 11  uob_alum                   15604 non-null  int32         
 12  home

In [49]:
dataframe.head()

,Department,Applicant ID,City - Address line 4,Programme,Latest Decision,Latest Decision Date,Country of Birth,Domicile,Nationality,Country of Institution,Grade on application,uob_alum,home_oversea,master_prev,distance,Institution,Institution_app_num,Fee Status,School,A1 Created date,A1 Updated date,A2 Created date,Decision Date,Response Date,Days at Admissions Review,Days in School,Days at Decision Stage,Total Days Admissions,TOTAL Days,Response days,decision_since_Sep01,A1C_since_Sep01,A1U_since_Sep01,A2C_since_Sep01,Response_since_Sep01,funding,Decision_C,Decision_R,Decision_U,Status,Qual_Name,Ielts_Overall,Ielts_Listening,Ielts_Reading,Ielts_Writing,Ielts_Speaking,Toefl_Overall,Toefl_Listening,Toefl_Reading,Toefl_Writing,Toefl_Speaking,PTE_Overall,PTE_Listening,PTE_Reading,PTE_Writing,PTE_Speaking,eng_type,multi_app,Prog Code,Prog Desc,Date Started,offered,offer_num,accepted,Institution_offer_num,Insti_weight
0,COMPUTER SCI,2305538,BENGALURU,MSc Computer Science FT,Conditional Offer - waiting for response,2021-02-25,NaN,NaN,NaN,IndiaIndia,8.4,0,1,0,8000.0,PES University,12.0,O,Computer Science,2020-11-30,2021-01-28,2021-02-05,2021-02-25,NaT,59.0,8.0,20.0,79.0,87.0,0.0,177.0,90.0,149.0,157.0,0.0,0.0,1.0,0.0,0.0,Active,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,1,NaN,NaN,NaN,1,1.0,0,9.0,0.750000
1,BUSINESS,2376697,KHAJRANA INDORE,MSc Adv Eng Man (Constr Man)FT,Conditional Offer - waiting for response,2021-06-10,India,India,India,IndiaIndia,7.42 CGPA,0,1,0,8000.0,ACROPOLIS INSTITUTE OF TECHNOLOGY & RESEARCH,1.0,O,Business School,2021-05-24,2021-06-10,2021-02-05,2021-06-10,NaT,17.0,0.0,0.0,17.0,17.0,0.0,282.0,265.0,282.0,0.0,0.0,0.0,1.0,0.0,0.0,Active,IELTS (International English Language Testing ...,8.0,8.0,9.0,6.5,7.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,1,NaN,NaN,NaN,1,1.0,0,1.0,1.000000
2,COMPUTER SCI,2374445,"BARWAHA , KHARGONE",MSc Artific Int & Mach Lear FT,Rejected at stage 1,2021-06-25,India,India,India,IndiaIndia,73.50%,0,1,0,8000.0,"AISSM'S Institute of Information Technology, Pune",1.0,O,Computer Science,2021-05-12,2021-06-25,2021-02-05,2021-06-25,NaT,44.0,0.0,0.0,44.0,44.0,0.0,297.0,253.0,297.0,0.0,0.0,0.0,0.0,1.0,0.0,Active,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,1,NaN,NaN,NaN,0,NaN,0,NaN,NaN
3,EEC ENG,2263436,CHANGCHUN CITY,MSc Elec + Computer Eng FT,Withdrawn,2020-11-06,China,China,China,China,79.13%,0,1,0,8000.0,Guizhou University,5.0,O,Elec Elec and System Eng;Elec Elec and Sys Eng...,2020-10-06,2020-11-06,2021-02-05,2020-11-06,2006-11-20,31.0,0.0,0.0,31.0,31.0,0.0,66.0,35.0,66.0,0.0,66.0,0.0,1.0,0.0,0.0,Withdrawn,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,1,NaN,NaN,NaN,1,1.0,0,5.0,1.000000
4,COMPUTER SCI,2295129,"WEIHAI, SHANDONG",MSc Artific Int & Mach Lear FT,Conditional Offer - waiting for response,2021-03-18,China,China,China,ChinaChina,,0,1,0,8000.0,Shandong University,9.0,O,Computer Science,2020-11-16,2021-02-15,2021-02-15,2021-03-18,2006-11-20,91.0,0.0,31.0,122.0,122.0,0.0,198.0,76.0,167.0,167.0,0.0,0.0,1.0,0.0,0.0,Active,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,2,NaN,NaN,NaN,1,2.0,0,4.0,0.444444


## **8. Save dataframe for the further research**

In [54]:
# only pick the application with a offer, to study whether the applicant will accept
offer = dataframe.loc[dataframe["offered"] == 1]

In [52]:
dataframe.to_pickle("df/data_clean.pkl") 
offer.to_pickle("df/offer_holder.pkl") 

In [42]:
offer = pd.read_pickle("df/offer_holder.pkl") 

In [43]:
offer.shape

(8912, 66)

In [44]:
dataframe = pd.read_pickle("df/data_clean.pkl") 
dataframe.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 15604 entries, 0 to 15603
Data columns (total 66 columns):
 #   Column                     Non-Null Count  Dtype         
---  ------                     --------------  -----         
 0   Department                 15604 non-null  object        
 1   Applicant ID               15604 non-null  int64         
 2   City - Address line 4      15604 non-null  object        
 3   Programme                  15604 non-null  object        
 4   Latest Decision            15601 non-null  object        
 5   Latest Decision Date       15238 non-null  datetime64[ns]
 6   Country of Birth           15587 non-null  object        
 7   Domicile                   15531 non-null  object        
 8   Nationality                15587 non-null  object        
 9   Country of Institution     15570 non-null  object        
 10  Grade on application       15604 non-null  object        
 11  uob_alum                   15604 non-null  int64         
 12  home

In [55]:
import os
print(os.path.abspath('.'))


C:\Users\Hedwig\Latest


Save to csv is also okay for temporary storage but it would generate one more index column. Therefore pickle is better. The following code is no longer used\

In [65]:
offer.to_csv('offer_holder.csv')

In [66]:
dataframe.to_csv('data_clean.csv')

dataframe = pd.read_csv("data_clean.csv")
dataframe.head()